In [ ]:
import pandas as pd
import numpy as np

X = pd.read_parquet("/content/2018_flights_cleaned_final_before_merging.parquet")

In [ ]:
X.drop(["FlightDate"], axis=1, inplace=True)

In [ ]:
X.to_parquet("2018_flights_cleaned_final_before_merging.parquet")

In [ ]:
del(X)

In [ ]:
!pip install duckdb


In [ ]:
import duckdb
con = duckdb.connect(database='flight_weather.duckdb', read_only=False)

In [ ]:
# Load Parquet and CSVs directly
con.execute("""
CREATE or replace TABLE flights AS
SELECT * FROM '/content/2018_flights_cleaned_final_before_merging.parquet';
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE weather AS
SELECT *,
       regexp_extract(filename, '([^/\\\\]+)\\.csv$', 1) AS airport_code
FROM read_csv_auto('*.csv');
""").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,4697994


In [ ]:
con.execute("""CREATE OR REPLACE TABLE flight_weather AS
WITH origin_weather AS (
    SELECT
        f.__index_level_0__,  -- or whatever unique identifier you have
        f.Origin,
        f.Dep_DateTime,
        AVG(o.WND)  AS o_WND,
        AVG(o.TMP)  AS o_TMP,
        AVG(o.DEW)  AS o_DEW,
        AVG(o.SLP)  AS o_SLP,
        AVG(o.CIG)  AS o_CIG,
        AVG(o.VIS)  AS o_VIS,
        AVG(o.AA1)  AS o_AA1,
        AVG(o.AA2)  AS o_AA2,
        MODE(o.AT1) AS o_AT1,
        MODE(o.AT2) AS o_AT2,
        MODE(o.AU1) AS o_AU1,
        MODE(o.AU2) AS o_AU2,
        MODE(o.AW1) AS o_AW1,
        MODE(o.AW2) AS o_AW2,
        MODE(o.GD1) AS o_GD1,
        MODE(o.GD2) AS o_GD2,
        AVG(o.OC1)  AS o_OC1
    FROM flights f
    LEFT JOIN weather o
      ON f.Origin = o.airport_code
     AND o.DATE BETWEEN f.Dep_DateTime - INTERVAL 6 HOUR AND f.Dep_DateTime + INTERVAL 6 HOUR
    GROUP BY f.__index_level_0__, f.Origin, f.Dep_DateTime
),
dest_weather AS (
    SELECT
        f.__index_level_0__,
        f.Dest,
        f.Arr_DateTime,
        AVG(d.WND)  AS d_WND,
        AVG(d.TMP)  AS d_TMP,
        AVG(d.DEW)  AS d_DEW,
        AVG(d.SLP)  AS d_SLP,
        AVG(d.CIG)  AS d_CIG,
        AVG(d.VIS)  AS d_VIS,
        AVG(d.AA1)  AS d_AA1,
        AVG(d.AA2)  AS d_AA2,
        MODE(d.AT1) AS d_AT1,
        MODE(d.AT2) AS d_AT2,
        MODE(d.AU1) AS d_AU1,
        MODE(d.AU2) AS d_AU2,
        MODE(d.AW1) AS d_AW1,
        MODE(d.AW2) AS d_AW2,
        MODE(d.GD1) AS d_GD1,
        MODE(d.GD2) AS d_GD2,
        AVG(d.OC1)  AS d_OC1
    FROM flights f
    LEFT JOIN weather d
      ON f.Dest = d.airport_code
     AND d.DATE BETWEEN f.Arr_DateTime - INTERVAL 6 HOUR AND f.Arr_DateTime + INTERVAL 6 HOUR
    GROUP BY f.__index_level_0__, f.Dest, f.Arr_DateTime
)

SELECT
    f.*,
    o.* EXCLUDE (__index_level_0__, Origin, Dep_DateTime),
    d.* EXCLUDE (__index_level_0__, Dest, Arr_DateTime)
FROM flights f
LEFT JOIN origin_weather o ON f.__index_level_0__ = o.__index_level_0__
LEFT JOIN dest_weather d   ON f.__index_level_0__ = d.__index_level_0__;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import os
from glob import glob
import shutil

In [ ]:
weather_folder = "/content"
base_flight_file = "/content/2018_flights_cleaned_final_before_merging.parquet"
working_flight_file = "flights_weather_current.parquet"

# Start by copying base dataset as working file
if not os.path.exists(working_flight_file):
    shutil.copy(base_flight_file, working_flight_file)

weather_files = sorted(glob(f"{weather_folder}/*.csv"))

In [ ]:
for i, wf in enumerate(weather_files, 1):
    airport_code = os.path.basename(wf).split('.')[0].upper()
    print(f"\nProcessing {airport_code} ({i}/{len(weather_files)})...")

    # Register the working flights parquet and current weather CSV
    con.execute(f"CREATE OR REPLACE TABLE flights AS SELECT * FROM '{working_flight_file}';")
    con.execute(f"CREATE OR REPLACE TABLE weather AS SELECT *, '{wf.split('.')[0].split('/')[2]}' as airport_code FROM '{wf}';")

    print("read")

    merged_df = con.execute(f"""
    SELECT
    f.*,

    -- ===== ORIGIN WEATHER (mean for numeric, mode for categorical) =====
    AVG(w.WND)  AS o_WND,
    AVG(w.CIG)  AS o_CIG,
    AVG(w.VIS)  AS o_VIS,
    AVG(w.TMP)  AS o_TMP,
    AVG(w.DEW)  AS o_DEW,
    AVG(w.SLP)  AS o_SLP,
    AVG(w.AA1)  AS o_AA1,
    AVG(w.AA2)  AS o_AA2,
    MODE(w.AT1) AS o_AT1,
    MODE(w.AT2) AS o_AT2,
    MODE(w.AU1) AS o_AU1,
    MODE(w.AU2) AS o_AU2,
    MODE(w.AW1) AS o_AW1,
    MODE(w.AW2) AS o_AW2,
    MODE(w.GD1) AS o_GD1,
    MODE(w.GD2) AS o_GD2,
    AVG(w.OC1)  AS o_OC1,

    -- ===== DESTINATION WEATHER =====
    AVG(wd.WND)  AS d_WND,
    AVG(wd.CIG)  AS d_CIG,
    AVG(wd.VIS)  AS d_VIS,
    AVG(wd.TMP)  AS d_TMP,
    AVG(wd.DEW)  AS d_DEW,
    AVG(wd.SLP)  AS d_SLP,
    AVG(wd.AA1)  AS d_AA1,
    AVG(wd.AA2)  AS d_AA2,
    MODE(wd.AT1) AS d_AT1,
    MODE(wd.AT2) AS d_AT2,
    MODE(wd.AU1) AS d_AU1,
    MODE(wd.AU2) AS d_AU2,
    MODE(wd.AW1) AS d_AW1,
    MODE(wd.AW2) AS d_AW2,
    MODE(wd.GD1) AS d_GD1,
    MODE(wd.GD2) AS d_GD2,
    AVG(wd.OC1)  AS d_OC1

FROM flights f

-- ===== ORIGIN JOIN =====
LEFT JOIN weather w
  ON f.Origin = w.airport_code
 AND w.DATE BETWEEN f.Dep_DateTime - INTERVAL 6 HOUR AND f.Dep_DateTime + INTERVAL 6 HOUR

-- ===== DESTINATION JOIN =====
LEFT JOIN weather wd
  ON f.Dest = wd.airport_code
 AND wd.DATE BETWEEN f.Arr_DateTime - INTERVAL 6 HOUR AND f.Arr_DateTime + INTERVAL 6 HOUR

GROUP BY ALL;
""").df()

    print("merged")
        # Save result to a temp parquet
    temp_out = f"flights_weather_{wf.split('.')[0].split('/')[2]}.parquet"
    merged_df.to_parquet(temp_out, index=False)

    # Delete old working file and rename new one
    os.remove(working_flight_file)
    os.rename(temp_out, working_flight_file)

    print(f"Updated working file with {airport_code} weather.")

print("\n🎉 All weather files integrated!")
print(f"Final merged dataset → {working_flight_file}")


Processing ABE (1/362)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

read


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

merged
Updated working file with ABE weather.

Processing ABI (2/362)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

read


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
import os
import duckdb
import pandas as pd
from glob import glob

# --- Setup paths ---
con = duckdb.connect()
weather_folder = "/content"
base_flight_file = "/content/2018_flights_cleaned_final_before_merging.parquet"

origin_dir = "/content/merged_batches_origin"
dest_dir = "/content/merged_batches_dest"
os.makedirs(origin_dir, exist_ok=True)
os.makedirs(dest_dir, exist_ok=True)

weather_files = sorted(glob(f"{weather_folder}/*.csv"))

# =======================================================
# PASS 1: ORIGIN WEATHER
# # =======================================================
# for i, wf in enumerate(weather_files, 1):
#     airport_code = os.path.splitext(os.path.basename(wf))[0].upper()
#     print(f"\n[ORIGIN] Processing {airport_code} ({i}/{len(weather_files)})...")

#     # Load only flights where this airport is the ORIGIN
#     flights_subset = con.execute(f"""
#         SELECT *
#         FROM '{base_flight_file}'
#         WHERE Origin = '{airport_code}'
#     """).df()

#     if flights_subset.empty:
#         print(f"No origin flights for {airport_code}, skipping.")
#         continue

#     # Register tables
#     con.register("flights", flights_subset)
#     con.execute(f"""
#         CREATE OR REPLACE TABLE weather AS
#         SELECT *, '{airport_code}' AS airport_code FROM '{wf}';
#     """)

#     # Join with ±6h window on departure time
#     merged_df = con.execute("""
#         SELECT
#             f.*,
# AVG(CAST(w.WND AS DOUBLE))  AS o_WND,
# AVG(CAST(w.CIG AS DOUBLE))  AS o_CIG,
# AVG(CAST(w.VIS AS DOUBLE))  AS o_VIS,
# AVG(CAST(w.TMP AS DOUBLE))  AS o_TMP,
# AVG(CAST(w.DEW AS DOUBLE))  AS o_DEW,
# AVG(CAST(w.SLP AS DOUBLE))  AS o_SLP,
# AVG(CAST(w.AA1 AS DOUBLE))  AS o_AA1,
# AVG(CAST(w.AA2 AS DOUBLE))  AS o_AA2,
# MODE(w.AT1) AS o_AT1,
# MODE(w.AT2) AS o_AT2,
# MODE(w.AU1) AS o_AU1,
# MODE(w.AU2) AS o_AU2,
# MODE(w.AW1) AS o_AW1,
# MODE(w.AW2) AS o_AW2,
# MODE(w.GD1) AS o_GD1,
# MODE(w.GD2) AS o_GD2,
# AVG(CAST(w.OC1 AS DOUBLE))  AS o_OC1
#         FROM flights f
#         LEFT JOIN weather w
#           ON w.airport_code = f.Origin
#          AND w.DATE BETWEEN f.Dep_DateTime - INTERVAL 6 HOUR AND f.Dep_DateTime + INTERVAL 6 HOUR
#         GROUP BY ALL;
#     """).df()

#     merged_df.to_parquet(f"{origin_dir}/{airport_code}_origin.parquet", index=False)
#     print(f"✅ Saved origin batch: {airport_code}_origin.parquet")

# =======================================================
# PASS 2: DESTINATION WEATHER
# =======================================================
for i, wf in enumerate(weather_files, 1):
    airport_code = os.path.splitext(os.path.basename(wf))[0].upper()
    print(f"\n[DEST] Processing {airport_code} ({i}/{len(weather_files)})...")

    # Load only flights where this airport is the DESTINATION
    flights_subset = con.execute(f"""
        SELECT *
        FROM '{base_flight_file}'
        WHERE Dest = '{airport_code}'
    """).df()

    if flights_subset.empty:
        print(f"No destination flights for {airport_code}, skipping.")
        continue

    # Register tables
    con.register("flights", flights_subset)
    con.execute(f"""
        CREATE OR REPLACE TABLE weather AS
        SELECT *, '{airport_code}' AS airport_code FROM '{wf}';
    """)

    # Join with ±6h window on arrival time
    merged_df = con.execute("""
        SELECT
            f.*,
            AVG(CAST(wd.WND AS DOUBLE))  AS d_WND,
AVG(CAST(wd.CIG AS DOUBLE))  AS d_CIG,
AVG(CAST(wd.VIS AS DOUBLE))  AS d_VIS,
AVG(CAST(wd.TMP AS DOUBLE))  AS d_TMP,
AVG(CAST(wd.DEW AS DOUBLE))  AS d_DEW,
AVG(CAST(wd.SLP AS DOUBLE))  AS d_SLP,
AVG(CAST(wd.AA1 AS DOUBLE))  AS d_AA1,
AVG(CAST(wd.AA2 AS DOUBLE))  AS d_AA2,
MODE(wd.AT1) AS d_AT1,
MODE(wd.AT2) AS d_AT2,
MODE(wd.AU1) AS d_AU1,
MODE(wd.AU2) AS d_AU2,
MODE(wd.AW1) AS d_AW1,
MODE(wd.AW2) AS d_AW2,
MODE(wd.GD1) AS d_GD1,
MODE(wd.GD2) AS d_GD2,
AVG(CAST(wd.OC1 AS DOUBLE))  AS d_OC1
        FROM flights f
        LEFT JOIN weather wd
          ON wd.airport_code = f.Dest
         AND wd.DATE BETWEEN f.Arr_DateTime - INTERVAL 6 HOUR AND f.Arr_DateTime + INTERVAL 6 HOUR
        GROUP BY ALL;
    """).df()

    merged_df.to_parquet(f"{dest_dir}/{airport_code}_dest.parquet", index=False)
    print(f"✅ Saved destination batch: {airport_code}_dest.parquet")

print("\n🎉 All origin and destination weather merges complete!")



[DEST] Processing ABE (1/362)...
✅ Saved destination batch: ABE_dest.parquet

[DEST] Processing ABI (2/362)...
✅ Saved destination batch: ABI_dest.parquet

[DEST] Processing ABQ (3/362)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved destination batch: ABQ_dest.parquet

[DEST] Processing ABR (4/362)...
✅ Saved destination batch: ABR_dest.parquet

[DEST] Processing ABY (5/362)...
✅ Saved destination batch: ABY_dest.parquet

[DEST] Processing ACK (6/362)...
✅ Saved destination batch: ACK_dest.parquet

[DEST] Processing ACT (7/362)...
✅ Saved destination batch: ACT_dest.parquet

[DEST] Processing ACV (8/362)...
✅ Saved destination batch: ACV_dest.parquet

[DEST] Processing ACY (9/362)...
✅ Saved destination batch: ACY_dest.parquet

[DEST] Processing ADK (10/362)...
✅ Saved destination batch: ADK_dest.parquet

[DEST] Processing ADQ (11/362)...
✅ Saved destination batch: ADQ_dest.parquet

[DEST] Processing AEX (12/362)...
✅ Saved destination batch: AEX_dest.parquet

[DEST] Processing AGS (13/362)...
✅ Saved destination batch: AGS_dest.parquet

[DEST] Processing AKN (14/362)...
✅ Saved destination batch: AKN_dest.parquet

[DEST] Processing ALB (15/362)...
✅ Saved destination batch: ALB_dest.parquet

[DEST] Proces

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved destination batch: ANC_dest.parquet

[DEST] Processing APN (20/362)...
✅ Saved destination batch: APN_dest.parquet

[DEST] Processing ART (21/362)...
✅ Saved destination batch: ART_dest.parquet

[DEST] Processing ASE (22/362)...
✅ Saved destination batch: ASE_dest.parquet

[DEST] Processing ATL (23/362)...


...

✅ Saved destination batch: SYR_dest.parquet

[DEST] Processing TLH (339/362)...
✅ Saved destination batch: TLH_dest.parquet

[DEST] Processing TOL (340/362)...
✅ Saved destination batch: TOL_dest.parquet

[DEST] Processing TPA (341/362)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved destination batch: TPA_dest.parquet

[DEST] Processing TRI (342/362)...
✅ Saved destination batch: TRI_dest.parquet

[DEST] Processing TTN (343/362)...
✅ Saved destination batch: TTN_dest.parquet

[DEST] Processing TUL (344/362)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved destination batch: TUL_dest.parquet

[DEST] Processing TUS (345/362)...
✅ Saved destination batch: TUS_dest.parquet

[DEST] Processing TVC (346/362)...
✅ Saved destination batch: TVC_dest.parquet

[DEST] Processing TWF (347/362)...
✅ Saved destination batch: TWF_dest.parquet

[DEST] Processing TXK (348/362)...
✅ Saved destination batch: TXK_dest.parquet

[DEST] Processing TYR (349/362)...
✅ Saved destination batch: TYR_dest.parquet

[DEST] Processing TYS (350/362)...
✅ Saved destination batch: TYS_dest.parquet

[DEST] Processing UIN (351/362)...
✅ Saved destination batch: UIN_dest.parquet

[DEST] Processing USA (352/362)...
✅ Saved destination batch: USA_dest.parquet

[DEST] Processing VEL (353/362)...
✅ Saved destination batch: VEL_dest.parquet

[DEST] Processing VLD (354/362)...
✅ Saved destination batch: VLD_dest.parquet

[DEST] Processing VPS (355/362)...
✅ Saved destination batch: VPS_dest.parquet

[DEST] Processing WRG (356/362)...
✅ Saved destination batch: WRG_dest.parq

In [ ]:
import shutil
from google.colab import files

# Folder you want to download
folder_path = "/content/merged_batches_dest"

# Create a ZIP of the folder
shutil.make_archive("merged_batches_dest", 'zip', folder_path)

# Download it
files.download("merged_batches_dest.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import duckdb

# Connect to DuckDB
# con = duckdb.connect(database=':memory:')

# Read and merge all origin files into one big table
con.execute("""
    CREATE TABLE origin AS
    SELECT * FROM read_parquet('/content/origin/*.parquet');
""")

# Read and merge all destination files into another big table
con.execute("""
    CREATE TABLE dest AS
    SELECT * FROM read_parquet('/content/destination/*.parquet');
""")
print("read")

# Now join both on FlightID
merged_df = con.execute("""
    SELECT
        o.FlightID,
        o.*,
        d.d_WND, d.d_CIG, d.d_VIS, d.d_TMP, d.d_DEW, d.d_SLP,
        d.d_AA1, d.d_AA2, d.d_AT1, d.d_AT2, d.d_AU1, d.d_AU2,
        d.d_AW1, d.d_AW2, d.d_GD1, d.d_GD2, d.d_OC1
    FROM origin o
    LEFT JOIN dest d USING (FlightID);
""").df()

# Save the fully merged dataset
merged_df.to_parquet("/content/flights_2018_final_merged.parquet", index=False)

print("✅ Final dataset saved → flights_2018_final_merged.parquet")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

read


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Final dataset saved → flights_2018_final_merged.parquet


In [ ]:
len(merged_df)

5531151

In [ ]:
merged_df.head(1000).to_csv("final_merged_1000_rows.csv")

In [ ]:
df = merged_df.sort_values(by="FlightID", ascending=True)
df.reset_index(drop=True, inplace=True)
df

,FlightID,Origin,Dest,CRSDepTime,CRSArrTime,Distance,Month,DayofMonth,DayOfWeek,IATA_Code_Marketing_Airline,...,d_AA2,d_AT1,d_AT2,d_AU1,d_AU2,d_AW1,d_AW2,d_GD1,d_GD2,d_OC1
0,0,ABY,ATL,1202,1304,145.0,1,23,2,DL,...,21.10,16.0,13.0,NaN,NaN,NaN,NaN,2.0,2.0,10.625
1,1,ABY,ATL,1202,1304,145.0,1,24,3,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,9.800
2,2,ABY,ATL,1202,1304,145.0,1,25,4,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
3,3,ABY,ATL,1202,1304,145.0,1,26,5,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN
4,4,ABY,ATL,1400,1500,145.0,1,27,6,DL,...,0.25,16.0,13.0,1.0,NaN,61.0,NaN,1.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531146,5531146,SCE,IAD,1445,1546,133.0,9,11,2,UA,...,2.55,16.0,14.0,0.0,0.0,10.0,51.0,4.0,4.0,NaN
5531147,5531147,IAD,GSO,1235,1355,239.0,9,11,2,UA,...,NaN,16.0,14.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN
5531148,5531148,EVV,ORD,1030,1204,272.0,9,11,2,UA,...,NaN,13.0,NaN,0.0,NaN,10.0,NaN,0.0,NaN,NaN
5531149,5531149,ORD,HPN,1410,1726,738.0,9,11,2,UA,...,0.20,14.0,1.0,0.0,0.0,10.0,51.0,5.0,3.0,NaN


In [ ]:
merged_df.columns

Index(['FlightID', 'Origin', 'Dest', 'CRSDepTime', 'CRSArrTime', 'Distance',
       'Month', 'DayofMonth', 'DayOfWeek', 'IATA_Code_Marketing_Airline',
       'Tail_Number', 'Dep_DateTime', 'Arr_DateTime', 'FlightID_1',
       '__index_level_0__', 'o_WND', 'o_CIG', 'o_VIS', 'o_TMP', 'o_DEW',
       'o_SLP', 'o_AA1', 'o_AA2', 'o_AT1', 'o_AT2', 'o_AU1', 'o_AU2', 'o_AW1',
       'o_AW2', 'o_GD1', 'o_GD2', 'o_OC1', 'd_WND', 'd_CIG', 'd_VIS', 'd_TMP',
       'd_DEW', 'd_SLP', 'd_AA1', 'd_AA2', 'd_AT1', 'd_AT2', 'd_AU1', 'd_AU2',
       'd_AW1', 'd_AW2', 'd_GD1', 'd_GD2', 'd_OC1'],
      dtype='object')

In [ ]:
del(merged_df)

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
Y = pd.read_parquet("/content/target_variables.parquet")

In [ ]:
len(Y)

5531151

In [ ]:
len(df)

5531151

In [ ]:
Y.columns

Index(['Cancelled', 'DepDelay', 'ArrDelay'], dtype='object')

In [ ]:
y_dep = Y["DepDelay"].copy()
y_arr = Y["ArrDelay"].copy()
y_cancel = Y["Cancelled"].copy()

In [ ]:
del(Y)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531151 entries, 0 to 5531150
Data columns (total 49 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   FlightID                     int64         
 1   Origin                       object        
 2   Dest                         object        
 3   CRSDepTime                   int64         
 4   CRSArrTime                   int64         
 5   Distance                     float64       
 6   Month                        int64         
 7   DayofMonth                   int64         
 8   DayOfWeek                    int64         
 9   IATA_Code_Marketing_Airline  object        
 10  Tail_Number                  object        
 11  Dep_DateTime                 datetime64[ns]
 12  Arr_DateTime                 datetime64[ns]
 13  FlightID_1                   int64         
 14  __index_level_0__            int64         
 15  o_WND                        float64       
 16  

In [ ]:
df = pd.read_parquet("/content/flights_2018_final_merged.parquet")

In [ ]:
cat_cols = [
    "IATA_Code_Marketing_Airline",
    "Origin",
    "Dest",
    "Tail_Number",
    # Weather categorical columns
    "o_AT1", "o_AT2", "o_AU1", "o_AU2", "o_AW1", "o_AW2", "o_GD1", "o_GD2",
    "d_AT1", "d_AT2", "d_AU1", "d_AU2", "d_AW1", "d_AW2", "d_GD1", "d_GD2"
]

# -----------------------------
# 2. Convert all categorical columns to string dtype
# -----------------------------
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].astype("string")

# --------------------------

# Optionally save back
df.to_parquet("/content/flights_2018_final_encoded.parquet", index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5531151 entries, 0 to 5531150
Data columns (total 49 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   FlightID                     int64         
 1   Origin                       string        
 2   Dest                         string        
 3   CRSDepTime                   int64         
 4   CRSArrTime                   int64         
 5   Distance                     float64       
 6   Month                        int64         
 7   DayofMonth                   int64         
 8   DayOfWeek                    int64         
 9   IATA_Code_Marketing_Airline  string        
 10  Tail_Number                  string        
 11  Dep_DateTime                 datetime64[ns]
 12  Arr_DateTime                 datetime64[ns]
 13  FlightID_1                   int64         
 14  __index_level_0__            int64         
 15  o_WND                        float64       
 16  